In [30]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [31]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [32]:
team_names = ["ohio-state","michigan", "michigan-state", "maryland", "purdue", "nebraska", "minnesota", "wisconsin", "penn-state", "iowa", "illinois", "rutgers", "indiana", "northwestern"]

stats_urls = [f"https://www.sports-reference.com/cfb/schools/{team}/2021.html" for team in team_names]
rosters_urls = [f"https://www.sports-reference.com/cfb/schools/{team}/2021-roster.html" for team in team_names]

In [33]:
wd = webdriver.Chrome('chromedriver', options=options)

In [34]:
# list of sub dataframes from each team which will later be concatenated 
passing = []
rushing_receiving = []
defense = []
scoring = []

In [35]:
index = 0

# iterates over both lists (stats_urls and rosters_urls)
for (stats_site, roster_site) in zip(stats_urls, rosters_urls):
    wd.get(stats_site)
    html_stats = wd.page_source
    stats = pd.read_html(html_stats) # stats contains all of the tables where stats[0] is the first table (current injuries)
    
    wd.get(roster_site)
    html_roster = wd.page_source
    roster = pd.read_html(html_roster) # roster[0] is the roster table 
    
    # assigns dataframes to their corresponding categories of stats
    # removes filler rows 'NaN' and 'Player' which occur when the table has a break (remove rows where the Rank is not a number)
    if len(stats) == 8: # accounts for 8 tables (has 'Current Injuries' table)
        passing_df = stats[2][pd.to_numeric(stats[2]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
        rushing_receiving_df = stats[3][pd.to_numeric(stats[3]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
        defense_df = stats[4][pd.to_numeric(stats[4]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
        scoring_df = stats[7][pd.to_numeric(stats[7]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
    
    # accounts for 7 tables (does not have 'Current Injuries' table)
    else:
        passing_df = stats[1][pd.to_numeric(stats[1]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
        rushing_receiving_df = stats[2][pd.to_numeric(stats[2]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
        defense_df = stats[3][pd.to_numeric(stats[3]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
        scoring_df = stats[6][pd.to_numeric(stats[6]['Unnamed: 0_level_0']['Rk'], errors='coerce').notnull()]
    
    # position of each player
    position = []
    
    # finds position of each player that is named in the 'passing' dataframe
    for player in passing_df['Unnamed: 1_level_0']['Player']:
        ind = roster[0][roster[0]['Player'] == player].index.values[0] # finds index of the player in the roster
        pos = roster[0].Pos[ind] # uses index to get position of the player
        position.append(pos)
        
    # adds columns 'Position' and 'School' to the passing stats dataframe
    passing_df['Position'] = position
    passing_df['School'] = team_names[index]
    
    # adds the sub dataframe of passing stats to the list 
    passing.append(passing_df)
    
    
    
    # REPEAT --> find position of each player that is named in the 'rushing_receiving' dataframe
    position.clear()
    
    for player in rushing_receiving_df['Unnamed: 1_level_0']['Player']:
        ind = roster[0][roster[0]['Player'] == player].index.values[0] # finds index of the player in the roster
        pos = roster[0].Pos[ind] # uses index to get position of the player
        position.append(pos)
        
    # adds columns 'Position' and 'Scool' to the 'rushing_receiving' dataframe
    rushing_receiving_df['Position'] = position
    rushing_receiving_df['School'] = team_names[index]
    
    # adds the sub dataframe of rushing_receiving to the list
    rushing_receiving.append(rushing_receiving_df)
    
    
    
    # REPEAT --> find position of each player that is named in the 'defense' dataframe
    position.clear()
    
    # account for rows that have 'NaN' or 'Player' in the Player column --> this is when the table has a break
    for player in defense_df['Unnamed: 1_level_0']['Player']: 
        ind = roster[0][roster[0]['Player'] == player].index.values[0] # finds index of the player in the roster
        pos = roster[0].Pos[ind] # uses index to get position of the player
        position.append(pos)
        
    # adds columns 'Position' and 'School' to the 'defense' dataframe
    defense_df['Position'] = position
    defense_df['School'] = team_names[index]
    
    # adds the sub dataframe of defense to the list
    defense.append(defense_df)    
    
    
    
    # REPEAT --> find position of each player that is named in the 'scoring' dataframe
    position.clear()
    
    # account for rows that have 'NaN' or 'Player' in the Player column --> this is when the table has a break
    for player in scoring_df['Unnamed: 1_level_0']['Player']: 
        ind = roster[0][roster[0]['Player'] == player].index.values[0] # finds index of the player in the roster
        pos = roster[0].Pos[ind] # uses index to get position of the player
        position.append(pos)
        
    # adds columns 'Position' and 'School' to the 'scoring' dataframe
    scoring_df['Position'] = position
    scoring_df['School'] = team_names[index]
    
    # adds the sub dataframe of 'scoring' to the list
    scoring.append(scoring_df) 
    
    # repeat for next team
    index += 1

In [36]:
# concatenate the list of dataframes for each category
passing_overall = pd.concat(passing, axis=0, ignore_index=True)
rushing_receiving_overall = pd.concat(rushing_receiving, axis=0, ignore_index=True)
defense_overall = pd.concat(defense, axis=0, ignore_index=True)
scoring_overall = pd.concat(scoring, axis=0, ignore_index=True)

In [37]:
# ranks the 'passing' stats by total number of yards
passing_overall.sort_values(by = [('Passing', 'Yds')], ascending = False, ignore_index = True, inplace = True)

In [38]:
# ranks the 'rushing/receiving' stats by total number of rushing yards
rushing_overall = rushing_receiving_overall.sort_values(by = [('Rushing', 'Yds')], ascending = False, ignore_index = True)

# ranks the 'rushing/receiving' stats by total number of receiving yards
receiving_overall = rushing_receiving_overall.sort_values(by = [('Receiving', 'Yds')], ascending = False, ignore_index = True)

In [39]:
# converts the 'Total Tackles,' 'Total Sacks,' and 'Total Passes Defended' columns from type object to type float
defense_overall = defense_overall.astype({('Tackles', 'Tot'):'float', ('Tackles', 'Sk'):'float', ('Def Int', 'PD'):'float'})

# ranks the 'defense' stats by total number of tackles
defense_tackles = defense_overall.sort_values(by = [('Tackles', 'Tot')], ascending = False, ignore_index = True)

# ranks the 'defense' stats by total number of sacks
defense_sacks = defense_overall.sort_values(by = [('Tackles', 'Sk')], ascending = False, ignore_index = True)

# ranks the 'defense' stats by total number passes defended
defense_pd = defense_overall.sort_values(by = [('Def Int', 'PD')], ascending = False, ignore_index = True)

In [40]:
# ranks the 'scoring' stats by total number points
scoring_overall.sort_values(by = [('Unnamed: 15_level_0', 'Pts')], ascending = False, ignore_index = True, inplace = True)

In [41]:
# remove the 'Rank' column from each dataframe that is assigned by each individual team
passing_overall.drop(('Unnamed: 0_level_0', 'Rk'), axis = 1, inplace = True)
rushing_overall.drop(('Unnamed: 0_level_0', 'Rk'), axis = 1, inplace = True)
receiving_overall.drop(('Unnamed: 0_level_0', 'Rk'), axis = 1, inplace = True)
defense_tackles.drop(('Unnamed: 0_level_0', 'Rk'), axis = 1, inplace = True)
defense_sacks.drop(('Unnamed: 0_level_0', 'Rk'), axis = 1, inplace = True)
defense_pd.drop(('Unnamed: 0_level_0', 'Rk'), axis = 1, inplace = True)
scoring_overall.drop(('Unnamed: 0_level_0', 'Rk'), axis = 1, inplace = True)

In [43]:
# write each of the dataframe stats to excel
writer = pd.ExcelWriter('output.xlsx') 

passing_overall.to_excel(writer, sheet_name = 'Overall Passing')
rushing_overall.to_excel(writer, sheet_name = 'Overall Rushing')
receiving_overall.to_excel(writer, sheet_name = 'Overall Receiving')
defense_tackles.to_excel(writer, sheet_name = 'Defensive Tackles')
defense_sacks.to_excel(writer, sheet_name = 'Defensive Sacks')
defense_pd.to_excel(writer, sheet_name = 'Passes Defended')
scoring_overall.to_excel(writer, sheet_name = 'Overall Scoring')



In [44]:
writer.save()